In [4]:
import numpy as np
import rasterio

#data_dir = "/home/cornelius/Documentos/Glaciar_segmentation_datasets/dataset_glacier/masks_data/LC08_L2SP_003070_20230819_20230825_02_T1_006_011.TIF"
data_dir = "/home/cornelius/Documentos/Glaciar_segmentation_datasets/dataset_glacier/SHOW_SIX_IMAGE/testing2/masks/LC08_L2SP_001071_20160817_20200906_02_T1_014_006.png"
#data_dir = "/home/cornelius/Documentos/Glaciar_segmentation_datasets/dataset_glacier/images_data_base/testing/masks/LC08_L2SP_001071_20160801_20200906_02_T1_014_005_flip_horizontal.TIF"
# Abrir el archivo raster
with rasterio.open(data_dir) as src:
    img = src.read(1)  # Leer la primera banda
    #img = img > 0.5

# Asegurarse de que la imagen sea binaria (0 y 255)
unique_values = np.unique(img)
print(f"Valores únicos en la imagen: {unique_values}")

# Si la imagen tiene valores 0 y 255, consideramos 255 como píxeles blancos
if set(unique_values).issubset({0, 255}):
    # Calcular el número de píxeles blancos (255)
    num_white_pixels = np.sum(img == 255)

    # Resolución espacial de la imagen (tamaño de píxel en metros)
    pixel_size = 30  # Por ejemplo, para Landsat 8 el tamaño del píxel es de 30x30 metros

    # Calcular el área en metros cuadrados
    area_m2 = num_white_pixels * (pixel_size ** 2)

    print(f'Área de la laguna: {area_m2} metros cuadrados')
else:
    print("La imagen no es completamente binaria.")

Valores únicos en la imagen: [  0   1   2   4   5   7  11  12  13  17  19  22  24  26  29  31  35  44
  51  88 119 124 128 129 138 142 152 154 155 160 177 181 191 193 199 200
 206 210 213 218 222 223 231 235 240 242 243 248 253 255]
La imagen no es completamente binaria.


/home/cornelius/miniconda3/envs/pytorch_watnet/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [5]:
import os
import rasterio
import numpy as np

input_dir = "/home/cornelius/Documentos/Glaciar_dataset_C2_L1/Analisis_temporal/USGS/Recortar_patchs_L5/Analisis_512x512_correct/"
output_dir = "/home/cornelius/Documentos/Glaciar_dataset_C2_L1/Analisis_temporal/USGS/Recortar_patchs_L5/Analisis_512x512_correct/TIF/"

def convert_binary(file_path, output_dir):
    # Abrir el archivo con rasterio
    with rasterio.open(file_path) as src:
        binary_img = src.read(1)

    # Umbralizar la imagen para obtener una imagen binaria
    threshold = 0.5
    mask_binary = np.where(binary_img > threshold, 1, 0).astype(np.uint8)

    # Obtener el nombre base del archivo original y cambiar la extensión a .TIF
    base_name = os.path.basename(file_path)
    mask_name = f"{os.path.splitext(base_name)[0]}.TIF"
    output_file_path = os.path.join(output_dir, mask_name)

    # Guardar la imagen binaria en formato .TIF
    save_img(mask_binary, output_file_path)

def save_img(mask, file_path):
    # Guardar la imagen binaria utilizando rasterio
    with rasterio.open(
        file_path,
        'w',
        driver='GTiff',
        height=mask.shape[0],
        width=mask.shape[1],
        count=1,
        dtype=np.uint8,
    ) as dst:
        dst.write(mask, 1)

# Iterar sobre los archivos en el directorio de entrada
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.tif', '.png')):  # Aceptar tanto .TIF como .png
        file_path = os.path.join(input_dir, filename)
        convert_binary(file_path, output_dir)

print("Máscaras guardadas en formato .TIF")


Máscaras guardadas en formato .TIF
